In [1]:
# -*- coding: utf-8 -*-
import math
import random

alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [2]:
# 根据传入的密钥，生成字母替换规则字典
# 例如传入"DGHJKL..."，生成字典{D:A, G:B, H:C...}
def create_cipher_dict(cipher):   # cipher密码
    cipher_dict = {}
    alphabet_list = list(alphabet)  # alphabet 字母
    for i in range(len(cipher)):
        cipher_dict[alphabet_list[i]] = cipher[i]
    return cipher_dict


In [3]:
# 使用密钥对文本进行替换(加密/解密)
def apply_cipher_on_text(text,cipher):  # 在文本上应用密码 加密
    cipher_dict = create_cipher_dict(cipher) 
    text = list(text)
    newtext = ""
    for elem in text:
        if elem.upper() in cipher_dict:  # upper() 小写字母转为大写字母
            newtext+=cipher_dict[elem.upper()]
        else:
            newtext+=" "
    return newtext

In [4]:
# 统计参考语料的bigram
# 例如 {'AB':234,'TH':2343,'CD':23 ..}
def create_scoring_params_dict(longtext_path):  # create_scoring_params_dict创建评分参数字典
    scoring_params = {}
    alphabet_list = list(alphabet)
    with open(longtext_path) as fp:
        for line in fp:
            data = list(line.strip())
            for i in range(len(data)-1):
                alpha_i = data[i].upper()
                alpha_j = data[i+1].upper()
                if alpha_i not in alphabet_list and alpha_i != " ":
                    alpha_i = " "
                if alpha_j not in alphabet_list and alpha_j != " ":
                    alpha_j = " "
                key = alpha_i+alpha_j
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
    return scoring_params

In [5]:
# 统计解密文本的bigram
# 例如 {'AB':234,'TH':2343,'CD':23 ..}
def score_params_on_cipher(text):
    scoring_params = {}
    alphabet_list = list(alphabet)
    data = list(text.strip())
    for i in range(len(data)-1):
        alpha_i =data[i].upper()
        alpha_j = data[i+1].upper()
        if alpha_i not in alphabet_list and alpha_i != " ":
            alpha_i = " "
        if alpha_j not in alphabet_list and alpha_j != " ":
            alpha_j = " "
        key = alpha_i+alpha_j
        if key in scoring_params:
            scoring_params[key]+=1
        else:
            scoring_params[key]=1
    return scoring_params

In [6]:
# 根据公式计算密钥的评分
def get_cipher_score(text,cipher,scoring_params):  #获取密码分数
    decrypted_text = apply_cipher_on_text(text,cipher)  #解密文本decrypted_text
    scored_f = score_params_on_cipher(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items(): 
        if k in scoring_params:
            cipher_score += v*math.log(scoring_params[k])
    return cipher_score

In [7]:
# 通过随机交换两个字母的顺序 生成一个新的密钥
def generate_cipher(cipher):
    pos1 = random.randint(0, len(list(cipher))-1)
    pos2 = random.randint(0, len(list(cipher))-1)
    if pos1 == pos2:
        return generate_cipher(cipher)
    else:
        cipher = list(cipher)
        pos1_alpha = cipher[pos1]
        pos2_alpha = cipher[pos2]
        cipher[pos1] = pos2_alpha
        cipher[pos2] = pos1_alpha
        return "".join(cipher)

In [8]:
# 抛一枚出现正面概率为p的硬币，出现正面返回True，出现反面返回False
def random_coin(p):
    unif = random.uniform(0,1)
    if unif>=p:
        return False
    else:
        return True

In [9]:
# MCMC方法解密 运行n_iter轮
def MCMC_decrypt(n_iter,cipher_text,scoring_params):
    current_cipher = alphabet # 以随机密钥开始
    state_keeper = set()
    best_state = ''
    score = 0
    for i in range(n_iter):
        state_keeper.add(current_cipher)
        proposed_cipher = generate_cipher(current_cipher)
        score_current_cipher = get_cipher_score(cipher_text,current_cipher,scoring_params)
        score_proposed_cipher = get_cipher_score(cipher_text,proposed_cipher,scoring_params)
        acceptance_probability = min(1,math.exp(score_proposed_cipher-score_current_cipher))
        if score_current_cipher>score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i%500==0:
            print("iter",i,":",apply_cipher_on_text(cipher_text,current_cipher)[0:99])
    return state_keeper,best_state

In [10]:
scoring_params = create_scoring_params_dict('war_and_peace.txt')
scoring_params #得分


{'TH': 73755,
 'HE': 75069,
 'E ': 104826,
 ' P': 16567,
 'PR': 7855,
 'RO': 14770,
 'OJ': 121,
 'JE': 564,
 'EC': 5400,
 'CT': 4134,
 'T ': 59070,
 ' G': 9133,
 'GU': 1340,
 'UT': 11010,
 'TE': 19134,
 'EN': 26370,
 'NB': 151,
 'BE': 11373,
 'ER': 44100,
 'RG': 1396,
 'G ': 21042,
 ' E': 10693,
 'EB': 254,
 'BO': 4095,
 'OO': 7196,
 'OK': 3381,
 'K ': 5171,
 ' O': 32237,
 'OF': 16919,
 'F ': 18279,
 ' W': 37905,
 'WA': 12162,
 'AR': 17836,
 'R ': 31043,
 ' A': 65084,
 'AN': 45652,
 'ND': 34571,
 'D ': 72007,
 'PE': 8205,
 'EA': 13337,
 'AC': 7019,
 'CE': 13136,
 '  ': 81107,
 ' B': 23146,
 'BY': 2550,
 'Y ': 30163,
 ' L': 13466,
 'LE': 16910,
 'EO': 1619,
 'O ': 25292,
 ' T': 83419,
 'TO': 23772,
 'OL': 7428,
 'LS': 1555,
 'ST': 17764,
 'OY': 710,
 'HI': 25958,
 'IS': 21595,
 'S ': 60012,
 ' I': 29235,
 ' F': 19702,
 'FO': 8091,
 'OR': 19773,
 ' U': 5201,
 'US': 8919,
 'SE': 17557,
 'NY': 2077,
 'YO': 5985,
 'ON': 27083,
 'NE': 14833,
 'YW': 81,
 'WH': 11925,
 'RE': 35653,
 'AT': 2815

In [11]:
# 测试文本

plain_text = "As Oliver gave this first proof of the free and proper action of his lungs, \
the patchwork coverlet which was carelessly flung over the iron bedstead, rustled; \
the pale face of a young woman was raised feebly from the pillow; and a faint voice imperfectly \
articulated the words, Let me see the child, and die. \
The surgeon had been sitting with his face turned towards the fire: giving the palms of his hands a warm \
and a rub alternately. As the young woman spoke, he rose, and advancing to the bed's head, said, with more kindness \
than might have been expected of him: "   #加密后的文本

encryption_key = "XEBPROHYAUFTIDSJLKZMWVNGQC"   # 加密密钥
cipher_text = apply_cipher_on_text(plain_text,encryption_key)  

decryption_key = "ICZNBKXGMPRQTWFDYEOLJVUAHS"   # 解密密钥

print("Text To Decode:「解密后cipher_text」", cipher_text, "\n")  

states,best_state = MCMC_decrypt(10000,cipher_text,scoring_params)  

print("\n", "Decoded Text:「加密后plain_text」",apply_cipher_on_text(cipher_text,best_state), "\n")

print("MCMC KEY FOUND:「加密密钥」",best_state)   

print("ACTUAL DECRYPTION KEY:「实际解密密钥」",decryption_key)  


Text To Decode:「解密后cipher_text」 XZ STAVRK HXVR MYAZ OAKZM JKSSO SO MYR OKRR XDP JKSJRK XBMASD SO YAZ TWDHZ  MYR JXMBYNSKF BSVRKTRM NYABY NXZ BXKRTRZZTQ OTWDH SVRK MYR AKSD ERPZMRXP  KWZMTRP  MYR JXTR OXBR SO X QSWDH NSIXD NXZ KXAZRP ORRETQ OKSI MYR JATTSN  XDP X OXADM VSABR AIJRKORBMTQ XKMABWTXMRP MYR NSKPZ  TRM IR ZRR MYR BYATP  XDP PAR  MYR ZWKHRSD YXP ERRD ZAMMADH NAMY YAZ OXBR MWKDRP MSNXKPZ MYR OAKR  HAVADH MYR JXTIZ SO YAZ YXDPZ X NXKI XDP X KWE XTMRKDXMRTQ  XZ MYR QSWDH NSIXD ZJSFR  YR KSZR  XDP XPVXDBADH MS MYR ERP Z YRXP  ZXAP  NAMY ISKR FADPDRZZ MYXD IAHYM YXVR ERRD RGJRBMRP SO YAI   

iter 0 : XZ STAVRK HXVR MYAZ OAKZM JKSSO SO MYR OKRR XDP JKSJRK XBMASD SO YAZ TWDHZ  MYR JXMBYNSKF BSVRKTRM 
iter 500 : HS OUIMER YHME TLIS WIRST GROOW OW TLE WREE HAN GROGER HBTIOA OW LIS UFAYS  TLE GHTBLCORK BOMERUET 
iter 1000 : OS APIVER DOVE THIS WIRST BRAAW AW THE WREE ONL BRABER OGTIAN AW HIS PUNDS  THE BOTGHCARK GAVERPET 
iter 1500 : AS OBIVER GAVE THIS LIRST PROOL OL THE LREE AND PROPE

In [12]:
scoring_params = create_scoring_params_dict('speeches.txt')
scoring_params

{' S': 11946,
 'SP': 681,
 'PE': 3145,
 'EE': 1826,
 'EC': 1990,
 'CH': 1769,
 'H ': 2780,
 '  ': 33295,
 ' T': 28865,
 'TH': 19896,
 'HA': 8646,
 'AN': 12238,
 'NK': 1055,
 'K ': 2498,
 ' Y': 4112,
 'YO': 3921,
 'OU': 9225,
 'U ': 3560,
 'SO': 3045,
 'O ': 10336,
 ' M': 6720,
 'MU': 499,
 'UC': 731,
 'AT': 7735,
 'T ': 22979,
 'S ': 16431,
 ' N': 3890,
 'NI': 1269,
 'IC': 2346,
 'CE': 1587,
 'E ': 35521,
 ' I': 15654,
 'IS': 4469,
 'SN': 256,
 'N ': 12640,
 ' H': 7717,
 'HE': 15484,
 ' A': 16271,
 'A ': 4855,
 ' G': 6142,
 'GR': 1336,
 'RE': 10263,
 'EA': 4358,
 'GU': 493,
 'UY': 331,
 'Y ': 12738,
 ' D': 5710,
 'DO': 3383,
 'OE': 333,
 'ES': 4559,
 'GE': 1848,
 'ET': 2337,
 ' F': 3876,
 'FA': 647,
 'AI': 1883,
 'IR': 1071,
 'R ': 7273,
 ' P': 5145,
 'PR': 1433,
 'SS': 1054,
 'IT': 6472,
 ' J': 1192,
 'JU': 594,
 'US': 2887,
 'ST': 4683,
 'NO': 4186,
 'OT': 3024,
 'ND': 7576,
 'D ': 13435,
 'I ': 6323,
 'AV': 2715,
 'VE': 7628,
 'TO': 7267,
 'TE': 4380,
 'EL': 2259,
 'LL': 5420,
 'L '

In [13]:
plain_text = "Telegram provides end-to-end encrypted calls and optional end-to-end encrypted \
chats between two online users on smartphone clients, whereas cloud chats use client-server/server-client \
encryption. Users can send text and voice messages, animated stickers, make voice and video calls, \
and share an unlimited number of images, documents, user locations, contacts, and music. "   #加密后的文本

encryption_key = "XEBPROHYAUFTIDSJLKZMWVNGQC"   # 加密密钥
cipher_text = apply_cipher_on_text(plain_text,encryption_key)  #apply_cipher_on_text(text,cipher)

decryption_key = "ICZNBKXGMPRQTWFDYEOLJVUAHS" # 解密密钥

print("Text To Decode:「解密后cipher_text」", cipher_text, "\n") 

states,best_state = MCMC_decrypt(10000,cipher_text,scoring_params)  

print("\n", "Decoded Text:「加密后plain_text」",apply_cipher_on_text(cipher_text,best_state), "\n")

print("MCMC KEY FOUND:「加密密钥」",best_state)    

print("ACTUAL DECRYPTION KEY:「实际解密密钥」",decryption_key)  


Text To Decode:「解密后cipher_text」 MRTRHKXI JKSVAPRZ RDP MS RDP RDBKQJMRP BXTTZ XDP SJMASDXT RDP MS RDP RDBKQJMRP BYXMZ ERMNRRD MNS SDTADR WZRKZ SD ZIXKMJYSDR BTARDMZ  NYRKRXZ BTSWP BYXMZ WZR BTARDM ZRKVRK ZRKVRK BTARDM RDBKQJMASD  WZRKZ BXD ZRDP MRGM XDP VSABR IRZZXHRZ  XDAIXMRP ZMABFRKZ  IXFR VSABR XDP VAPRS BXTTZ  XDP ZYXKR XD WDTAIAMRP DWIERK SO AIXHRZ  PSBWIRDMZ  WZRK TSBXMASDZ  BSDMXBMZ  XDP IWZAB   

iter 0 : MRTRHUXI JUSVAPRZ RDP MS RDP RDBUQJMRP BXTTZ XDP SJMASDXT RDP MS RDP RDBUQJMRP BYXMZ ERMNRRD MNS SD
iter 500 : TEDEVGAL UGOHIYES ERY TO ERY ERNGJUTEY NADDS ARY OUTIORAD ERY TO ERY ERNGJUTEY NCATS METWEER TWO OR
iter 1000 : TEDEVGAL UGOHIYES ERY TO ERY ERNGBUTEY NADDS ARY OUTIORAD ERY TO ERY ERNGBUTEY NCATS METWEER TWO OR
iter 1500 : TEDEVGAL UGOHIYES ERY TO ERY ERNGBUTEY NADDS ARY OUTIORAD ERY TO ERY ERNGBUTEY NCATS WETMEER TMO OR
iter 2000 : TEDEVGAL UGOHIYES ERY TO ERY ERNGBUTEY NADDS ARY OUTIORAD ERY TO ERY ERNGBUTEY NCATS WETMEER TMO OR
iter 2500 : TEDEVGAL UGOHIYES ERY TO